# Novartis Interview Questions
1. **How do you debug your spark code**
Mentioned about data skewness, data spillage and how to resolve them

    Follow up questions:

    i. **Repartition vs coalesce**

    ii. **Salting**

    iii. **Broadcast join**

    iv. **Caching**



2. **What can we infer from spark UI and how spark prepares it execution planning??**


3. **Why do we use window functions and name few of them??**


These function are used to perform calculations across a set of rows( a window) related to the current row.

| Function | Description |
| :-- | :-- |
| ROW_NUMBER() | Assigns a unique sequential number to each row within a partition. |
| RANK() | Assigns a rank to each row within a partition, with gaps for duplicate ranks. |
| DENSE_RANK() | Similar to RANK(), but without gaps in rank values for duplicate ranks. |
| NTILE(n) | Divides rows into n buckets and assigns a bucket number to each row. |
| SUM(column) | Computes the sum of values over the window. |
| AVG(column) | Computes the average of values over the window. |
| MIN(column) | Finds the minimum value over the window. |
| MAX(column) | Finds the maximum value over the window. |
| COUNT(column) | Counts the number of rows in the window. |
| FIRST_VALUE(column) | Returns the first value in the window. |
| LAST_VALUE(column) | Returns the last value in the window. |
| LEAD(column, n) | Returns the value of the column n rows ahead of the current row. |
| LAG(column, n) | Returns the value of the column n rows behind the current row. |
| CUME_DIST() | Computes the cumulative distribution of a value within a partition. |
| PERCENT_RANK() | Computes the relative rank of a row as a percentage of the total rows. |
| NTH_VALUE(column, n) | Returns the nth value in the window. |


4. **Default join spark uses**
    
| Join Strategy | When to Use | Advantages | Disadvantages |
| :-- | :-- | :-- | :-- |
| Broadcast Hash Join (BHJ) | One dataset is small enough to fit in memory. | Fast, avoids shuffling. | Limited by memory size. |
| Shuffle Hash Join (SHJ) | Both datasets are medium-sized and hashable. | Efficient for medium datasets. | Requires shuffling. |
| Sort-Merge Join (SMJ) | Both datasets are large and sortable. | Efficient for large datasets. | Sorting and shuffling are expensive. |
| Cartesian Join | No join condition (Cartesian product). | Useful for generating combinations. | Extremely expensive for large datasets. |
| Broadcast Nested Loop Join | Non-equijoin (e.g., > or <) with one small dataset. | Works for non-equijoin conditions. | Inefficient for large datasets. |
| Shuffle-and-Replicate NLJ | Non-equijoin with both large datasets. | Works for non-equijoin conditions. | Very expensive. |
| Skew Join | Data is highly skewed. | Handles skewed data efficiently. | Requires additional configuration. |


**Follow up questions:**

i. **how shuffling happens in sort-merge join, its advantages and disadvantages**

Steps:
- Partitioning: Data is partitioned based on the join keys. Spark uses a hash function to distribute rows with the same join key to the same partition across the cluster.
- Shuffling: Data is shuffled across the network to ensure that rows with the same join key co-located in the same partition. Consumes network I/O and time. 
- Sorting: Each partition is sorted by the join keys. Sorting ensures that rows with the same join key are grouped together, making the merge step efficient
- Merging: Sorted partitions are merged to produce the final result. Rows from both datasets are iterated over and matched based on the join key.

Advantages:
- Efficient for large datasets: SMJ is suitable for large datasets that can be sorted and merged efficiently.
- Optimized for euijoin conditions: It works well for equijoin conditions, where the join keys are equal.
- Handles Skewed Data Better: better than hash-based joins because sorting ensures that rows with the same key are grouped together.
- Default: when equi-join conditions are present.

Disadvantages:
- Expensive: Sorting and shuffling can be expensive in terms of time and resources, especially for large datasets.
- Memory Usage: Requires sufficient memory to hold the sorted partitions, which can lead to out-of-memory errors if the data is too large.
- Not Suitable for Non-Equijoin Conditions: SMJ is not suitable for non-equijoin conditions (e.g., >, <) as it relies on sorting based on equality.


5. **Scenarios when Driver gets OOM issues**

Mentioned about collect() and show() - They also gave hint about broadcast datasets also roots to OOM in driver and then I explained it


6. **Advantages of Parquet files compared to other file formats**


7. **How do you decide on number of executors when procssing huge volumes of data given huge cluster size** 
- Expalined about fat executor, thin executor concepts and given advantages
  and disadvantages of both of them and eventually said it depends on time/memory/resource constraints of the usecase
 

8. **What is small file issue in spark and how do we resolve them**
 - Mentioned about having less number of partition cols and combining multiple small files to a large file, optimize  
  in databricks


9. Get all categories with average price more than 1000$

product_table:
```csv
ID,name,category,price
1,Product A,Category 1,1500
2,Product B,Category 2,800
3,Product C,Category 1,1200
4,Product D,Category 3,2000
```


In [1]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("Manna").getOrCreate()
columns = ["ID","name","category","price"]
data = [
    (1, "Product A", "Category 1", 1500),
    (2, "Product B", "Category 2", 800),
    (3, "Product C", "Category 1", 1200),
    (4, "Product D", "Category 3", 2000)
]
spark.sparkContext.setJobDescription("Loading data from CSV file")
df = spark.createDataFrame(data, schema=columns)
df.createOrReplaceTempView("products")
spark.sparkContext.setJobDescription("Applying Transformation")
res=spark.sql("""
    select category
    from products group by category
    having avg(price) > 1000
""")
spark.sparkContext.setJobDescription("Showing Data")
res.show()
input("Press Enter to stop")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 12:11:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+
|  category|
+----------+
|Category 1|
|Category 3|
+----------+



''

10. **How can this below code be optimized?**

```python
user = (
	user.select("id", "name")
	.join(address.select("id", "country"), "id", "left")
	.filter(col("country") != "USA")
)
```

May be we can use `broadcast` join if address table is small enough to fit in memory.
Also, we can filter USAs before the join to reduce the data size being processed.

```python
user = (
	user.select("id", "name")
	.join(broadcast(address.select("id", "country").filter(col("country") != "USA")), "id", "left")
)
```


11. How many stages, jobs created as part of below code

```python
df = (
	df.select("appliance", "order_num")
	.repartition(5)
	.filter(col("order_num").isNotNull())
	.join(order_df.select("appliance", "category", "sub_category"), "appliance")
	.filter(col("sub_category") != "phone")
	.groupBy("category")
	.count(agg(collect_set("order_num")).alias("order_list"))
```

4. Question: 
Given the DOSE and TESTS tables, for each measurement in the TESTS table, assign the most recent dose (from the DOSE table) for the same subject_id where the dose time is less than or equal to the measurement time. If there is no such dose, leave the dose column blank.

**DOSES**
| subject_id |    time     | dose |
|:----------:|:----------:|:----:|
|    01      | 1751878800 |  5   |
|    01      | 1752052500 | 10   |
|    01      | 1752224160 | 15   |
|    02      | 1751878920 | 15   |
|    02      | 1752051900 | 12   |
|    02      | 1752224340 | 10   |

**TESTS**

| subject_id |    time     | mesurement |
|:----------:|:----------:|:----------:|
|    01      | 1751875000 |    2.5     |
|    01      | 1751954400 |    2.9     |
|    01      | 1752226200 |    2.7     |
|    02      | 1751880000 |    2.0     |
|    02      | 1752092600 |    2.0     |

**RESULT**

| subject_id |    time     | mesurement | dose |
|:----------:|:----------:|:----------:|:----:|
|    01      | 1751875000 |    2.5     |      |
|    01      | 1751954400 |    2.9     |  5   |
|    01      | 1752226200 |    2.7     | 15   |
|    02      | 1751880000 |    2.0     | 15   |
|    02      | 1752092600 |    2.0     | 12   |
